# Forex trading api for saas 

## development file 

In [1]:
import openai
import yfinance as yf
import pandas as pd 
import json
import time
from datetime import datetime, timedelta

In [2]:
api_key = 'OPEN_AI api key'
openai.api_key = api_key
client = openai.OpenAI()

## User accepted inputs 

- Time interval 
- Currency pair 

In [3]:
# Get historical data for EUR/USD
currency_pair = 'EURUSD=X'  # Yahoo Finance symbol for EUR/USD
data = yf.download(currency_pair, period='5d', interval='5m')

[*********************100%***********************]  1 of 1 completed


In [4]:
#Resizing the data
data = data.tail(49)

In [5]:
data['date'] = data.index
data.reset_index(inplace=True)
data_dict = data[["date","Open","High","Low","Close"]].to_dict()

C:\Users\Bongo\AppData\Local\Temp\ipykernel_11928\385119403.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = data.index


In [6]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": """ You are a professional trader with expertise in technical analysis. Your task is to analyze raw financial data (currency pair and price action dictionary) and provide a comprehensive trading recommendation, including:
- **Signal:** Clear buy or sell signal.
- **Entry Price:** Suggested price to open the trade.
- **Take Profit:** Suggested price to close the trade if profitable.
- **Stop Loss:** Suggested price to close the trade if not profitable.
**Input:**
- Currency Pair: {currency_pair}
- Price Action Dictionary: {data_dict}
**Output:**
Complete the following dictionary with your trading recommendations:
{"signal": "", "entry price": "", "take profit": "", "stop loss": ""}
**Note:**
- Base your recommendations on sound technical analysis principles.
- Prioritize accuracy in your response.
- Only provide the completed dictionary no other text
"""
        },
        {"role": "user", "content": f"Currency Pair: {currency_pair} and Dictionary: {data_dict}"}
    ]
)

In [7]:
#Getting the result
result = completion.choices[0].message.content

In [8]:
print(result)

{"signal": "Sell", "entry price": "1.107665", "take profit": "1.106500", "stop loss": "1.108770"}


## As a file

`import openai
import yfinance as yf
import pandas as pd
import json

api_key = 'OPEN_AI api key'
openai.api_key = api_key
client = openai.OpenAI()

def fetch_historical_data(currency_pair, period='5d', interval='5m'):
    data = yf.download(currency_pair, period=period, interval=interval)
    if data is None or data.empty:
        raise ValueError("Failed to fetch historical data or empty dataset.")
    
    # Resize and format the data
    data = data.tail(49)
    data['date'] = data.index
    data.reset_index(inplace=True)
    price_data_dict = data[["date", "Open", "High", "Low", "Close"]].to_dict()
    return price_data_dict
    
def extract_values_from_output(output):
    # Parse the output string as JSON
    output_json = json.loads(output)
    
    # Extract values and assign them to variables
    signal = output_json["signal"]
    entry_price = output_json["entry price"]
    take_profit = output_json["take profit"]
    stop_loss = output_json["stop loss"]
    
    return signal, entry_price, take_profit, stop_loss

def get_trading_recommendation(currency_pair, data_dict):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": """ Your task is to provide a trading recommendation based on:
- **Signal:** Clear buy or sell signal.
- **Entry Price:** Suggested price to open the trade.
- **Take Profit:** Suggested price to close the trade if profitable.
- **Stop Loss:** Suggested price to close the trade if not profitable.
**Input:**
- Currency Pair: {currency_pair}
- Price Action Dictionary: {data_dict}
**Output:**
Complete the following dictionary with your trading recommendations:
{"signal": "", "entry price": "", "take profit": "", "stop loss": ""}
"""
            },
            {"role": "user", "content": f"Currency Pair: {currency_pair} and Dictionary: {data_dict}"}
        ]
    )
    return completion.choices[0].message.content

def main():
    currency_pair = 'EURUSD=X'
    
    try:
        price_data_dict = fetch_historical_data(currency_pair)
        trading_recommendation = get_trading_recommendation(currency_pair, price_data_dict)
        signal, entry_price, take_profit, stop_loss = extract_values_from_output(trading_recommendation)
        return print(f"{signal} {entry_price} {take_profit} {stop_loss}")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()
`

## Functions 

In [9]:
def get_trading_recommendation(currency_pair, data_dict):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": """ Your task is to provide a trading recommendation based on:
- **Signal:** Clear buy or sell signal.
- **Entry Price:** Suggested price to open the trade.
- **Take Profit:** Suggested price to close the trade if profitable.
- **Stop Loss:** Suggested price to close the trade if not profitable.
**Input:**
- Currency Pair: {currency_pair}
- Price Action Dictionary: {data_dict}
**Output:**
Complete the following dictionary with your trading recommendations:
{"signal": "", "entry price": "", "take profit": "", "stop loss": ""}
"""
            },
            {"role": "user", "content": f"Currency Pair: {currency_pair} and Dictionary: {data_dict}"}
        ]
    )
    return completion.choices[0].message.content

In [10]:
def extract_values_from_output(output):
    try:
        if not output:
            # Handle the case where output is empty or None
            return None, None, None, None
        
        # Parse the output string as JSON
        output_json = json.loads(output)
        
        # Extract values and assign them to variables
        signal = output_json.get("signal")
        entry_price = output_json.get("entry price")
        take_profit = output_json.get("take profit")
        stop_loss = output_json.get("stop loss")
        
        return signal, entry_price, take_profit, stop_loss
    
    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}")
        return None, None, None, None

In [11]:
def fetch_historical_data(currency_pair, period='5d', interval='5m'):
    data = yf.download(currency_pair, period=period, interval=interval)
    if data is None or data.empty:
        raise ValueError("Failed to fetch historical data or empty dataset.")
    
    # Resize and format the data
    data = data.tail(49)
    data['date'] = data.index
    data.reset_index(inplace=True)
    price_data_dict = data[["date", "Open", "High", "Low", "Close"]].to_dict()
    return price_data_dict

## Backtest  baby! 

In [12]:
# Function to fetch historical data from yfinance
def fetch_historical_data_backtest(currency_pair, period='1d', interval='5m'):
    data = yf.download(currency_pair, period=period, interval=interval)
    data['date'] = data.index
    data.reset_index(inplace=True)
    data = data[["date", "Open", "High", "Low", "Close"]]
    return data

In [13]:
# Function to perform backtesting
def perform_backtest(currency_pair):
    # Fetch historical data for backtesting
    backtest_data = fetch_historical_data_backtest(currency_pair)
    
    # Initialize columns for profit/loss and order information
    backtest_data['Profit/Loss'] = 0.0
    backtest_data['Order'] = ''
    backtest_data['Order Price'] = 0.0
    backtest_data['Take Profit'] = 0.0
    backtest_data['Stop Loss'] = 0.0
    
    # Perform backtesting logic
    for i in range(32, len(backtest_data)):
        if backtest_data.at[i, 'Order'] == '':
            price_data_dict = backtest_data.iloc[i-32:i].to_dict()
            
            # Get trading recommendation from the model
            trading_recommendation = get_trading_recommendation(currency_pair, price_data_dict)
            
            
            # Extract values from the model output
            signal, entry_price, take_profit, stop_loss = extract_values_from_output(trading_recommendation)
            # Pause the code for 3 seconds
            time.sleep(20.5)
            
            backtest_data.at[i, 'Order'] = signal
            backtest_data.at[i, 'Order Price'] = entry_price
            backtest_data.at[i, 'Take Profit'] = take_profit
            backtest_data.at[i, 'Stop Loss'] = stop_loss
        
        # Check if conditions for closing the order are met
        elif backtest_data.at[i, 'Order'] == 'Buy':
            if backtest_data.at[i, 'High'] >= backtest_data.at[i, 'Take Profit'] or backtest_data.at[i, 'Low'] <= backtest_data.at[i, 'Stop Loss']:
                backtest_data.at[i, 'Profit/Loss'] = backtest_data.at[i, 'Close'] - backtest_data.at[i, 'Order Price']
                backtest_data.at[i+1, 'Order'] = ''
        
        elif backtest_data.at[i, 'Order'] == 'Sell':
            if backtest_data.at[i, 'Low'] <= backtest_data.at[i, 'Take Profit'] or backtest_data.at[i, 'High'] >= backtest_data.at[i, 'Stop Loss']:
                backtest_data.at[i, 'Profit/Loss'] = backtest_data.at[i, 'Order Price'] - backtest_data.at[i, 'Close']
                backtest_data.at[i+1, 'Order'] = ''

    # Calculate cumulative profit/loss
    backtest_data['Cumulative P/L'] = backtest_data['Profit/Loss'].cumsum()

    # Calculate win/loss ratio
    total_trades = len(backtest_data[backtest_data['Profit/Loss'] != 0])
    win_trades = len(backtest_data[backtest_data['Profit/Loss'] > 0])
    loss_trades = len(backtest_data[backtest_data['Profit/Loss'] < 0])
    win_loss_ratio = win_trades / loss_trades if loss_trades > 0 else win_trades
    
    return backtest_data, win_loss_ratio

In [14]:
# Perform the backtest
currency_pair = 'EURUSD=X'
backtest_results, win_loss_ratio = perform_backtest(currency_pair)

[*********************100%***********************]  1 of 1 completed


C:\Users\Bongo\AppData\Local\Temp\ipykernel_11928\1158014403.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  backtest_data['Profit/Loss'] = 0.0
C:\Users\Bongo\AppData\Local\Temp\ipykernel_11928\1158014403.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  backtest_data['Order'] = ''


JSON Decode Error: Unterminated string starting at: line 1 column 86 (char 85)
JSON Decode Error: Unterminated string starting at: line 1 column 99 (char 98)
JSON Decode Error: Expecting ',' delimiter: line 1 column 96 (char 95)
JSON Decode Error: Expecting property name enclosed in double quotes: line 1 column 55 (char 54)
JSON Decode Error: Unterminated string starting at: line 1 column 34 (char 33)
JSON Decode Error: Unterminated string starting at: line 1 column 19 (char 18)
JSON Decode Error: Unterminated string starting at: line 1 column 65 (char 64)
JSON Decode Error: Expecting value: line 1 column 1 (char 0)


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-K5VMIdA99vxb3T256OvOM8mD on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 7m12s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:
# Display backtest results
print("Backtest Results:")
print(backtest_results.tail())

In [ ]:
# Display win/loss ratio
print(f"Win/Loss Ratio: {win_loss_ratio}")